In [ ]:
# Master Setup
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Core imports
from core.combined_outlook import CombinedFiscalOutlookModel
from core.policies import PolicyType, get_policy_config
from core.simulation import run_comprehensive_simulation
from core.healthcare import simulate_healthcare_years
from core.social_security import SocialSecurityModel
from core.revenue_modeling import FederalRevenueModel
from core.monte_carlo_scenarios import MonteCarloEngine

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ All modules loaded!")
print(f"📅 Analysis Date: {datetime.now().strftime('%B %d, %Y')}")
print(f"📁 Project Root: {project_root}")

---

# Part 1: Baseline Analysis

## Current Fiscal Trajectory

First, establish where we're headed without policy changes:

In [ ]:
# Generate baseline projection
print("🔄 Generating 30-year baseline projection...")

model = CombinedFiscalOutlookModel()
baseline = model.project_outlook(2024, 2054)

years = list(range(2024, 2055))

print("\n📊 Baseline Summary:")
print("=" * 60)

# Key metrics at intervals
checkpoints = [0, 5, 10, 20, 30]  # Years from start
print(f"{'Year':<8} {'Debt ($T)':<12} {'Deficit ($B)':<14} {'Debt/GDP':>10}")
print("-" * 60)

for cp in checkpoints:
    if cp < len(years):
        y = years[cp]
        debt = baseline['total_debt'][cp] / 1e12
        deficit = baseline['annual_deficit'][cp] / 1e9
        debt_gdp = baseline.get('debt_to_gdp_ratio', [120]*31)[cp]
        print(f"{y:<8} ${debt:<10.2f}T ${deficit:<12.0f}B {debt_gdp:>9.1f}%")

In [ ]:
# Visualize baseline trajectory
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Total Debt
ax1 = axes[0, 0]
debt_t = [d/1e12 for d in baseline['total_debt']]
ax1.fill_between(years, debt_t, alpha=0.3, color='red')
ax1.plot(years, debt_t, 'r-', linewidth=2)
ax1.axhline(y=35, color='gray', linestyle='--', label='Current ~$35T')
ax1.set_xlabel('Year')
ax1.set_ylabel('$ Trillions')
ax1.set_title('Total Federal Debt')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Annual Deficit
ax2 = axes[0, 1]
deficit_b = [d/1e9 for d in baseline['annual_deficit']]
ax2.bar(years, deficit_b, color='orange', alpha=0.7)
ax2.axhline(y=1800, color='gray', linestyle='--', label='Current ~$1.8T')
ax2.set_xlabel('Year')
ax2.set_ylabel('$ Billions')
ax2.set_title('Annual Budget Deficit')
ax2.legend()

# 3. Revenue vs Spending
ax3 = axes[1, 0]
revenue_t = [r/1e12 for r in baseline['total_revenue']]
spending_t = [s/1e12 for s in baseline.get('total_spending', [r + d for r, d in zip(baseline['total_revenue'], baseline['annual_deficit'])])]
ax3.plot(years, revenue_t, 'g-', linewidth=2, label='Revenue')
ax3.plot(years, spending_t, 'r-', linewidth=2, label='Spending')
ax3.fill_between(years, revenue_t, spending_t, alpha=0.2, color='red')
ax3.set_xlabel('Year')
ax3.set_ylabel('$ Trillions')
ax3.set_title('Revenue vs Spending Gap')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Debt-to-GDP Ratio
ax4 = axes[1, 1]
debt_gdp = baseline.get('debt_to_gdp_ratio', [120 + i*2 for i in range(31)])
ax4.fill_between(years, debt_gdp, alpha=0.3, color='purple')
ax4.plot(years, debt_gdp, 'purple', linewidth=2)
ax4.axhline(y=100, color='red', linestyle='--', label='100% threshold')
ax4.axhline(y=77, color='green', linestyle='--', label='Historical avg')
ax4.set_xlabel('Year')
ax4.set_ylabel('Percent')
ax4.set_title('Debt-to-GDP Ratio')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle('30-Year Baseline Fiscal Outlook', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n⚠️  Key Concerns:")
print(f"   • Debt grows from ${debt_t[0]:.1f}T to ${debt_t[-1]:.1f}T ({(debt_t[-1]/debt_t[0]-1)*100:.0f}% increase)")
print(f"   • Debt-to-GDP ratio reaches {debt_gdp[-1]:.0f}% (vs historical ~77%)")
print(f"   • Annual deficit grows from ${deficit_b[0]:.0f}B to ${deficit_b[-1]:.0f}B")

---

# Part 2: Policy Selection & Comparison

Let's compare three major policy approaches:

In [ ]:
# Define comparison policies
policies_to_compare = [
    {
        'name': 'Status Quo',
        'description': 'Current law baseline',
        'color': 'gray',
        'annual_impact_bn': 0
    },
    {
        'name': 'Deficit Reduction',
        'description': 'Balanced approach: revenue + spending',
        'color': 'green',
        'revenue_changes': [
            ('Corporate Tax (28%)', 180),
            ('Carbon Tax', 200),
            ('Capital Gains Parity', 120),
        ],
        'spending_changes': [
            ('Drug Price Negotiation', -100),
            ('Defense Efficiency', -50),
            ('Administrative Savings', -30),
        ],
        'annual_impact_bn': 680
    },
    {
        'name': 'Progressive Investment',
        'description': 'Higher taxes, expanded programs',
        'color': 'blue',
        'revenue_changes': [
            ('Wealth Tax (2%)', 300),
            ('Corporate Tax (35%)', 350),
            ('Financial Transaction Tax', 150),
        ],
        'spending_changes': [
            ('Universal Pre-K', 100),
            ('Infrastructure', 200),
            ('R&D Investment', 80),
        ],
        'annual_impact_bn': 420  # Net deficit reduction
    },
    {
        'name': 'Fiscal Conservative',
        'description': 'Spending cuts, limited revenue',
        'color': 'red',
        'revenue_changes': [
            ('Base Broadening', 100),
        ],
        'spending_changes': [
            ('Discretionary Cuts (-15%)', -250),
            ('Entitlement Reform', -200),
            ('Block Grant Conversion', -100),
        ],
        'annual_impact_bn': 650
    }
]

print("📋 Policies to Compare:")
print("=" * 60)
for p in policies_to_compare:
    print(f"\n{p['name']}")
    print(f"   {p['description']}")
    print(f"   Net annual impact: ${p['annual_impact_bn']:+.0f}B deficit reduction")

In [ ]:
# Simulate all policies
def simulate_policy(policy_config, baseline, years):
    """Simulate policy impact over time."""
    annual_impact = policy_config['annual_impact_bn'] * 1e9
    
    policy_debt = []
    policy_deficit = []
    cumulative = 0
    
    for i, year in enumerate(years):
        if year >= 2025:  # Policy starts 2025
            cumulative += annual_impact
        
        adj_deficit = baseline['annual_deficit'][i] - (annual_impact if year >= 2025 else 0)
        policy_deficit.append(adj_deficit)
        policy_debt.append(baseline['total_debt'][i] - cumulative)
    
    return {
        'debt': policy_debt,
        'deficit': policy_deficit,
        'cumulative_savings': cumulative
    }

# Run all simulations
results = {}
for policy in policies_to_compare:
    results[policy['name']] = simulate_policy(policy, baseline, years)
    results[policy['name']]['color'] = policy['color']

print("✅ All policy simulations complete!")

# Summary table
print("\n📊 30-Year Policy Comparison:")
print("=" * 70)
print(f"{'Policy':<25} {'2054 Debt ($T)':<18} {'Cumulative Savings':>18}")
print("-" * 70)

for name, res in results.items():
    final_debt = res['debt'][-1] / 1e12
    savings = res['cumulative_savings'] / 1e12
    print(f"{name:<25} ${final_debt:<16.1f}T ${savings:>16.1f}T")

In [ ]:
# Comparative visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Debt trajectories
ax1 = axes[0]
for name, res in results.items():
    debt_t = [d/1e12 for d in res['debt']]
    ax1.plot(years, debt_t, color=res['color'], linewidth=2.5, label=name)

ax1.set_xlabel('Year')
ax1.set_ylabel('Total Debt ($ Trillions)')
ax1.set_title('Debt Trajectory by Policy')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Final debt comparison (bar chart)
ax2 = axes[1]
names = list(results.keys())
final_debts = [results[n]['debt'][-1]/1e12 for n in names]
colors = [results[n]['color'] for n in names]

bars = ax2.bar(names, final_debts, color=colors, alpha=0.7, edgecolor='black')
ax2.set_ylabel('2054 Debt ($ Trillions)')
ax2.set_title('Final Debt by Policy Scenario')
ax2.axhline(y=baseline['total_debt'][-1]/1e12, color='gray', linestyle='--', label='Baseline')

# Add value labels
for bar, debt in zip(bars, final_debts):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'${debt:.1f}T', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

---

# Part 3: Monte Carlo Uncertainty Analysis

Policy outcomes depend on uncertain economic conditions:

In [ ]:
# Monte Carlo simulation for selected policy
print("🎲 Running Monte Carlo Analysis...")
print("   (This simulates economic uncertainty)")

np.random.seed(42)
n_simulations = 500

# Simulate with varying economic assumptions
mc_results = []

for i in range(n_simulations):
    # Random economic variations
    gdp_growth = np.random.normal(0.023, 0.008)  # Mean 2.3%, SD 0.8%
    inflation = np.random.normal(0.025, 0.005)   # Mean 2.5%, SD 0.5%
    interest_rate = np.random.normal(0.04, 0.01) # Mean 4%, SD 1%
    
    # Apply to 'Deficit Reduction' policy
    base_impact = 680e9  # Annual impact
    
    # Economic multipliers
    growth_effect = 1 + (gdp_growth - 0.023) * 5  # GDP affects revenue
    inflation_effect = 1 + (inflation - 0.025) * 2  # Inflation affects costs
    
    adjusted_impact = base_impact * growth_effect / inflation_effect
    
    # Project 30-year cumulative
    cumulative = adjusted_impact * 30
    final_debt = baseline['total_debt'][-1] - cumulative
    
    mc_results.append({
        'final_debt': final_debt / 1e12,
        'cumulative_savings': cumulative / 1e12
    })

# Analysis
final_debts_mc = [r['final_debt'] for r in mc_results]
savings_mc = [r['cumulative_savings'] for r in mc_results]

print(f"\n📊 Monte Carlo Results (n={n_simulations}):")
print("=" * 50)
print(f"   Final Debt (2054):")
print(f"      • Mean: ${np.mean(final_debts_mc):.1f}T")
print(f"      • 5th percentile: ${np.percentile(final_debts_mc, 5):.1f}T")
print(f"      • 95th percentile: ${np.percentile(final_debts_mc, 95):.1f}T")
print(f"\n   Cumulative Savings:")
print(f"      • Mean: ${np.mean(savings_mc):.1f}T")
print(f"      • Range: ${np.min(savings_mc):.1f}T - ${np.max(savings_mc):.1f}T")

In [ ]:
# Monte Carlo visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution of final debt
ax1 = axes[0]
ax1.hist(final_debts_mc, bins=30, color='steelblue', edgecolor='white', alpha=0.7)
ax1.axvline(np.mean(final_debts_mc), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(final_debts_mc):.1f}T')
ax1.axvline(np.percentile(final_debts_mc, 5), color='orange', linestyle=':', linewidth=2, label=f'5th %ile: ${np.percentile(final_debts_mc, 5):.1f}T')
ax1.axvline(np.percentile(final_debts_mc, 95), color='orange', linestyle=':', linewidth=2, label=f'95th %ile: ${np.percentile(final_debts_mc, 95):.1f}T')
ax1.set_xlabel('2054 Debt ($ Trillions)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Final Debt\n(Deficit Reduction Policy)')
ax1.legend()

# Distribution of savings
ax2 = axes[1]
ax2.hist(savings_mc, bins=30, color='green', edgecolor='white', alpha=0.7)
ax2.axvline(np.mean(savings_mc), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(savings_mc):.1f}T')
ax2.set_xlabel('Cumulative Savings ($ Trillions)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Cumulative Savings\n(30 Years)')
ax2.legend()

plt.tight_layout()
plt.show()

# Confidence interval
ci_low = np.percentile(final_debts_mc, 5)
ci_high = np.percentile(final_debts_mc, 95)
print(f"\n📈 90% Confidence Interval for 2054 Debt:")
print(f"   ${ci_low:.1f}T - ${ci_high:.1f}T")
print(f"\n💡 Interpretation: There's 90% confidence that the 2054 debt")
print(f"   will fall between ${ci_low:.1f}T and ${ci_high:.1f}T under this policy.")

---

# Part 4: Component Deep Dive

Analyze specific budget components:

In [ ]:
# Social Security analysis
print("👴 Social Security Deep Dive")
print("=" * 50)

ss_model = SocialSecurityModel()
ss_projection = ss_model.project(2024, 2054)

print(f"\n📊 Trust Fund Status:")
print(f"   • Current (2024): ${ss_projection.get('trust_fund_balance', [2800e9])[0]/1e12:.2f}T")
print(f"   • Projected depletion: ~{ss_projection.get('depletion_year', 2034)}")

# Healthcare projection
print("\n🏥 Healthcare Spending")
print("=" * 50)

healthcare = simulate_healthcare_years(PolicyType.CURRENT_LAW, years=30)
print(f"   • 2024 Medicare: ${healthcare.get('medicare', [800e9])[0]/1e9:.0f}B")
print(f"   • 2054 Medicare: ${healthcare.get('medicare', [800e9]*31)[-1]/1e9:.0f}B")
print(f"   • 30-year growth: {((healthcare.get('medicare', [800e9]*31)[-1]/healthcare.get('medicare', [800e9])[0])-1)*100:.0f}%")

---

# Part 5: Synthesis Report

Generate a professional summary:

In [ ]:
# Generate Executive Summary
print("="*70)
print("                    FISCAL POLICY ANALYSIS REPORT")
print("="*70)
print(f"\nGenerated: {datetime.now().strftime('%B %d, %Y %H:%M')}")
print(f"Analysis Period: 2024-2054 (30 years)")
print(f"Monte Carlo Simulations: {n_simulations}")

print("\n" + "="*70)
print("EXECUTIVE SUMMARY")
print("="*70)

print("\n📊 BASELINE OUTLOOK:")
print(f"   Under current law, federal debt grows from ${baseline['total_debt'][0]/1e12:.1f}T")
print(f"   to ${baseline['total_debt'][-1]/1e12:.1f}T by 2054, a {((baseline['total_debt'][-1]/baseline['total_debt'][0])-1)*100:.0f}% increase.")

print("\n📋 POLICY COMPARISON:")
for name, res in results.items():
    if name != 'Status Quo':
        savings = (baseline['total_debt'][-1] - res['debt'][-1]) / 1e12
        print(f"   • {name}: ${res['debt'][-1]/1e12:.1f}T debt (saves ${savings:.1f}T)")

print("\n🎯 KEY FINDINGS:")
print("   1. All policy scenarios reduce debt relative to baseline")
print("   2. Deficit Reduction approach provides balanced outcomes")
print("   3. Economic uncertainty creates ±$5-10T range in outcomes")
print("   4. Social Security trust fund depletion remains a concern")

print("\n⚠️  UNCERTAINTIES & LIMITATIONS:")
print("   • Economic growth assumptions drive revenue projections")
print("   • Healthcare cost growth highly uncertain")
print("   • Political feasibility not assessed")
print("   • Behavioral responses to policy changes not fully modeled")

print("\n💡 RECOMMENDATIONS:")
print("   1. Consider balanced approach combining revenue and spending")
print("   2. Address Social Security solvency as priority")
print("   3. Healthcare cost containment critical for long-term")
print("   4. Build in policy flexibility for economic conditions")

print("\n" + "="*70)
print("                         END OF REPORT")
print("="*70)

In [ ]:
# Final comprehensive visualization
fig = plt.figure(figsize=(16, 12))

# Create grid
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. All policy trajectories
ax1 = fig.add_subplot(gs[0, :])
for name, res in results.items():
    debt_t = [d/1e12 for d in res['debt']]
    ax1.plot(years, debt_t, color=res['color'], linewidth=2.5, label=name)
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Debt ($ Trillions)')
ax1.set_title('30-Year Debt Trajectory: Policy Comparison', fontsize=12, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. Monte Carlo distribution
ax2 = fig.add_subplot(gs[1, 0])
ax2.hist(final_debts_mc, bins=25, color='steelblue', edgecolor='white', alpha=0.7)
ax2.axvline(np.mean(final_debts_mc), color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('2054 Debt ($T)')
ax2.set_ylabel('Frequency')
ax2.set_title('Monte Carlo: Debt Distribution')

# 3. Final debt bar comparison
ax3 = fig.add_subplot(gs[1, 1])
final_debts = [results[n]['debt'][-1]/1e12 for n in names]
colors = [results[n]['color'] for n in names]
bars = ax3.barh(names, final_debts, color=colors, alpha=0.7, edgecolor='black')
ax3.set_xlabel('2054 Debt ($T)')
ax3.set_title('Final Debt by Policy')

# 4. Cumulative savings
ax4 = fig.add_subplot(gs[1, 2])
savings = [(baseline['total_debt'][-1] - results[n]['debt'][-1])/1e12 for n in names]
ax4.barh(names, savings, color=colors, alpha=0.7, edgecolor='black')
ax4.set_xlabel('Cumulative Savings ($T)')
ax4.set_title('30-Year Savings vs Baseline')

# 5. Revenue vs Spending gap
ax5 = fig.add_subplot(gs[2, 0:2])
revenue_t = [r/1e12 for r in baseline['total_revenue']]
spending_t = [s/1e12 for s in baseline.get('total_spending', [r + d for r, d in zip(baseline['total_revenue'], baseline['annual_deficit'])])]
ax5.fill_between(years, revenue_t, spending_t, alpha=0.3, color='red', label='Deficit')
ax5.plot(years, revenue_t, 'g-', linewidth=2, label='Revenue')
ax5.plot(years, spending_t, 'r-', linewidth=2, label='Spending')
ax5.set_xlabel('Year')
ax5.set_ylabel('$ Trillions')
ax5.set_title('Federal Revenue vs Spending')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Key metrics summary
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
summary_text = f"""KEY METRICS (2054)
─────────────────────
Baseline Debt:   ${baseline['total_debt'][-1]/1e12:.1f}T
Best Policy:     ${min(final_debts):.1f}T
Max Savings:     ${max(savings):.1f}T

MONTE CARLO (90% CI)
─────────────────────
Debt Range:
${np.percentile(final_debts_mc, 5):.1f}T - ${np.percentile(final_debts_mc, 95):.1f}T

Mean Savings: ${np.mean(savings_mc):.1f}T
"""
ax6.text(0.1, 0.9, summary_text, transform=ax6.transAxes, fontsize=11,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.suptitle('COMPREHENSIVE FISCAL POLICY ANALYSIS', fontsize=14, fontweight='bold', y=0.98)
plt.savefig(str(project_root / 'reports' / 'capstone_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Report visualization saved to reports/capstone_analysis.png")

---

# 🎓 Congratulations!

You've completed the PoliSim educational curriculum!

## What You've Accomplished

✅ Built comprehensive baseline fiscal projections  
✅ Compared multiple policy scenarios  
✅ Quantified uncertainty with Monte Carlo analysis  
✅ Generated professional-quality analysis and visualizations  
✅ Synthesized findings into an executive summary  

---

## Next Steps

1. **Explore** — Try different policy combinations
2. **Extend** — Add your own analysis modules
3. **Contribute** — Submit improvements to PoliSim
4. **Apply** — Use these skills for real policy analysis

---

## Resources

- **Documentation**: `/docs/` folder
- **API Reference**: `/docs/API_ENDPOINTS.md`
- **GitHub**: Report issues and contribute
- **Community**: Join discussions

---

*Thank you for learning with PoliSim!* 🎉